In [12]:
!pip install mem0ai openai chromadb langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.1 MB/s eta 0:00:00


In [23]:
from google.colab import userdata
import os
import warnings

# Ignore DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# set the OPENAI_API_KEY environment variable
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [24]:
# instantiate the memory

from mem0 import Memory

m = Memory()

In [25]:
# 1. Add: Store a memory from any unstructured text

result = m.add("My name is Rounak", user_id="rik", metadata={"category": "personal"})

In [26]:
result = m.add("My favorite team is FC Barcelona", user_id="rik", metadata={"category": "personal"})

In [27]:
all_memories = m.get_all()

print(all_memories)

[{'id': '2ce81e93-2cac-41e8-b2e6-ed256953dd66', 'memory': 'Favorite team is FC Barcelona', 'hash': '7be37d69a3821a23ccb5a7c475f4d68d', 'metadata': {'category': 'personal'}, 'created_at': '2024-10-09T03:15:29.131425-07:00', 'updated_at': None, 'user_id': 'rik'}, {'id': 'a6517f91-ade7-492c-be48-8661ad8c88ae', 'memory': 'Name is Rounak', 'hash': '797fef94b597a0159bc2ac21a020f95e', 'metadata': {'category': 'personal'}, 'created_at': '2024-10-09T03:15:18.620331-07:00', 'updated_at': None, 'user_id': 'rik'}]


## Connect with Chroma VectorDB

In [28]:
import os
from mem0 import Memory

config = {
    "vector_store": {
        "provider": "chroma",
        "config": {
            "collection_name": "test",
            "path": "db",
        }
    }
}

m = Memory.from_config(config)

In [29]:
m.add("My name is Rounak", user_id="rik", metadata={"category": "personal"})

{'message': 'ok'}

In [30]:
memories = m.get_all(user_id='rik')

In [31]:
memories

[{'id': '5953826b-9d26-4ad5-a542-00b357600328',
  'memory': 'Name is Rounak',
  'hash': '797fef94b597a0159bc2ac21a020f95e',
  'metadata': {'category': 'personal'},
  'created_at': '2024-10-09T03:04:01.986007-07:00',
  'updated_at': None,
  'user_id': 'rik'}]

## With Memory vs Without Memory

In [32]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [33]:
def without_memory_response(query):
  messages_without_memory = [
      (
          "system",
          "You are a helpful assistant.",
      ),
      ("human", query),
  ]
  ai_msg_without_memory = llm.invoke(messages_without_memory)
  return ai_msg_without_memory.content

while True:
    user_input = input("Ask a question (or type 'q' to quit): ")

    if user_input.lower() == 'q':
        print("Exiting the loop. Goodbye!")
        break

    response = without_memory_response(user_input)
    print("AI response:", response)
    print("--------------------------------------------------------")

Ask a question (or type 'q' to quit): My favorite team is barcelona. Where is this club?
AI response: FC Barcelona, commonly referred to as Barcelona, is a professional football club based in Barcelona, Catalonia, Spain. The club was founded in 1899 and is one of the most successful and popular football clubs in the world. They play their home matches at the Camp Nou, which is one of the largest stadiums in Europe.
--------------------------------------------------------
Ask a question (or type 'q' to quit): Name two players from this club
AI response: Sure! However, I need to know which club you're referring to in order to provide you with the names of two players. Please specify the club.
--------------------------------------------------------
Ask a question (or type 'q' to quit): q
Exiting the loop. Goodbye!


In [34]:
def with_memory_response(query,user_id):
  memories = m.get_all(user_id=user_id)
  memory_string = ""

  for memory in memories:
    memory_string += memory["memory"] + "\n"

  messages_without_memory = [
      (
          "system",
          f"You are a helpful assistant. Here is the user information {memory_string}"
      ),
      ("human", query),
  ]
  m.add(query, user_id=user_id, metadata={"category": "personal"})

  ai_msg_with_memory = llm.invoke(messages_without_memory)
  return ai_msg_with_memory.content

while True:
    user_input = input("Ask a question (or type 'q' to quit): ")

    if user_input.lower() == 'q':
        print("Exiting the loop. Goodbye!")
        break

    response = with_memory_response(query=user_input, user_id="rounak")
    print("AI response:", response)
    print("--------------------------------------------------------")

Ask a question (or type 'q' to quit): My favorite team is barcelona. Where is this club?


AI response: FC Barcelona, commonly known as Barcelona or Barça, is based in Barcelona, Spain. The club was founded in 1899 and plays its home matches at the Camp Nou, which is one of the largest and most iconic stadiums in the world. Barcelona is known for its rich history, success in various competitions, and its distinctive style of play.
--------------------------------------------------------
Ask a question (or type 'q' to quit): Name two players from this club
AI response: Two players from FC Barcelona are Lionel Messi and Robert Lewandowski.
--------------------------------------------------------
Ask a question (or type 'q' to quit): q
Exiting the loop. Goodbye!
